## Credit card Default Use Case

Acme Bank is an internationally renowned bank which has been in the Indian banking sector since the past two decades. In the last year, they have started offering a premium travel credit card in association with AirIndia. Typically, the credit card business has been a profitable one for Acme. However, the Acme travel card has not registered profit so far. This is a worrying trend for the CEO as the overall Acme portfolio is stagnant and they were hoping to find new customers by targeting a more premium clientele.
A closer evaluation of the balance sheet shows that an average of 20% revenue is being lost monthly due to credit card default. If this revenue leakage is stopped, then the CEO believes that this card could become one of the most profitable product offerings in the company's portfolio.
As part of a specialized analytics firm, you are called in to assess if the cardholders’ past data and history can be used to predict default. If one is able to predict default even a month before its' occurrence, significant revenue savings can be made
You are provided with a consolidated database of individual customers and their credit history (refer to the file sent by email, “acme_bank_data.csv”)
The data team has also provided you with a data dictionary (“acme_bank_data_dictionnary.csv”)

### 1. Import the dataset into R

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df=pd.read_csv('acme_bank_data_v2.csv')

In [ ]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0,689,0,0,0,0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,0,2,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0,1000,1000,1000,0,2000,1
2,3,90000.0,2,2,2,34,0,0,0,0,0,0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518,1500,1000,1000,1000,5000,0
3,4,50000.0,2,2,1,37,0,0,0,0,0,0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000,2019,1200,1100,1069,1000,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,0,0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000,36681,10000,9000,689,679,0


In [ ]:
df.describe()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
count,30000.000000,29872.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,29954.000000,29988.000000,2.998400e+04,29996.000000,29994.000000,29996.000000,30000.000000,3.000000e+04,30000.00000,30000.000000,30000.000000,30000.000000,30000.000000
mean,15000.500000,167410.607927,1.603733,1.853133,1.551867,35.485500,-0.016700,-0.133767,-0.166200,-0.220667,-0.266200,-0.291100,51224.648862,49165.453982,4.701809e+04,43262.566242,40314.996866,38868.813475,5663.580500,5.921163e+03,5225.68150,4826.076867,4799.387633,5215.502567,0.221200
std,8660.398374,129721.364116,0.489129,0.790349,0.521970,9.217904,1.123802,1.197186,1.196868,1.169139,1.133187,1.149988,73619.488423,71132.443959,6.935518e+04,64336.673193,60802.116583,59555.613569,16563.280354,2.304087e+04,17606.96147,15666.159744,15278.305679,17777.465775,0.415062
min,1.000000,10000.000000,1.000000,0.000000,0.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-165580.000000,-69777.000000,-1.572640e+05,-170000.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000
25%,7500.750000,50000.000000,1.000000,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,3558.250000,2984.750000,2.667750e+03,2324.500000,1763.250000,1256.000000,1000.000000,8.330000e+02,390.00000,296.000000,252.500000,117.750000,0.000000
50%,15000.500000,140000.000000,2.000000,2.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22383.000000,21194.500000,2.008950e+04,19051.000000,18104.500000,17068.000000,2100.000000,2.009000e+03,1800.00000,1500.000000,1500.000000,1500.000000,0.000000
75%,22500.250000,240000.000000,2.000000,2.000000,2.000000,41.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,67113.500000,64006.250000,6.016950e+04,54506.000000,50208.500000,49177.250000,5006.000000,5.000000e+03,4505.00000,4013.250000,4031.500000,4000.000000,0.000000
max,30000.000000,1000000.000000,2.000000,6.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,964511.000000,983931.000000,1.664089e+06,891586.000000,927171.000000,961664.000000,873552.000000,1.684259e+06,896040.00000,621000.000000,426529.000000,528666.000000,1.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          30000 non-null  int64  
 1   LIMIT_BAL                   29872 non-null  float64
 2   SEX                         30000 non-null  int64  
 3   EDUCATION                   30000 non-null  int64  
 4   MARRIAGE                    30000 non-null  int64  
 5   AGE                         30000 non-null  int64  
 6   PAY_0                       30000 non-null  int64  
 7   PAY_2                       30000 non-null  int64  
 8   PAY_3                       30000 non-null  int64  
 9   PAY_4                       30000 non-null  int64  
 10  PAY_5                       30000 non-null  int64  
 11  PAY_6                       30000 non-null  int64  
 12  BILL_AMT1                   29954 non-null  float64
 13  BILL_AMT2                   299

In [ ]:
df.isna().sum()

ID                              0
LIMIT_BAL                     128
SEX                             0
EDUCATION                       0
MARRIAGE                        0
AGE                             0
PAY_0                           0
PAY_2                           0
PAY_3                           0
PAY_4                           0
PAY_5                           0
PAY_6                           0
BILL_AMT1                      46
BILL_AMT2                      12
BILL_AMT3                      16
BILL_AMT4                       4
BILL_AMT5                       6
BILL_AMT6                       4
PAY_AMT1                        0
PAY_AMT2                        0
PAY_AMT3                        0
PAY_AMT4                        0
PAY_AMT5                        0
PAY_AMT6                        0
default payment next month      0
dtype: int64

In [ ]:
"""for i in range(0,30000):
    if df.loc[[i]].isna().sum().sum()>2:
        df.drop([5,6], axis=0, inplace=True)"""

2426
9460
11748
14539


In [5]:
df.iloc[2426]

ID                             2427.0
LIMIT_BAL                     30000.0
SEX                               2.0
EDUCATION                         3.0
MARRIAGE                          1.0
AGE                              67.0
PAY_0                             1.0
PAY_2                             2.0
PAY_3                             0.0
PAY_4                             0.0
PAY_5                             0.0
PAY_6                             0.0
BILL_AMT1                         NaN
BILL_AMT2                     28092.0
BILL_AMT3                         NaN
BILL_AMT4                     29555.0
BILL_AMT5                         NaN
BILL_AMT6                     29252.0
PAY_AMT1                          0.0
PAY_AMT2                       1800.0
PAY_AMT3                       1200.0
PAY_AMT4                       1083.0
PAY_AMT5                       1200.0
PAY_AMT6                       1250.0
default payment next month        0.0
Name: 2426, dtype: float64

In [6]:
sns.pairplot(df, hue="default payment next month", diag_kind="hist")

KeyboardInterrupt: 